In [2]:
import numpy as np
from tinygrad.helpers import Timing

In [3]:
from tinygrad import Tensor

In [4]:
t1 = Tensor([1,2,3,4,5])
na = np.array([1,2,3,4,5])
t2 = Tensor(na)

In [8]:
full = Tensor.full(shape=(2,3), fill_value=5)
full.numpy()

array([[5, 5, 5],
       [5, 5, 5]], dtype=int32)

In [12]:
zeros = Tensor.zeros(2,3)
print(zeros.numpy())
ones = Tensor.ones(2,3)
print(ones.numpy())

[[0. 0. 0.]
 [0. 0. 0.]]
[[1. 1. 1.]
 [1. 1. 1.]]


In [15]:
full_like = Tensor.full_like(full, fill_value=2)
print(full_like.numpy())
zeros_like = Tensor.zeros_like(full)
print(zeros_like.numpy())
ones_like = Tensor.ones_like(full)
print(ones_like.numpy())

[[2 2 2]
 [2 2 2]]
[[0 0 0]
 [0 0 0]]
[[1 1 1]
 [1 1 1]]


In [17]:
eye = Tensor.eye(3)
print(eye.numpy())
arange = Tensor.arange(start=0, stop=10, step=1)
print(arange.numpy())

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[0 1 2 3 4 5 6 7 8 9]


In [20]:
rand = Tensor.rand(2, 3)
print(rand.numpy())
randn = Tensor.randn(2, 3)
print(randn.numpy())
uniform = Tensor.uniform(2, 3, low=0, high=10)
print(uniform.numpy())

[[0.55006063 0.9794905  0.6698835 ]
 [0.50536907 0.20245743 0.34587526]]
[[-1.0196397  -0.21291831  1.0557469 ]
 [ 1.2318832  -2.3998892   0.54259   ]]
[[4.468341  1.5921855 1.211685 ]
 [3.7904835 5.9335613 1.611501 ]]


In [24]:
from tinygrad import dtypes

t3 = Tensor([1, 2, 3, 4, 5], dtype=dtypes.int32)
t3.numpy()

array([1, 2, 3, 4, 5], dtype=int32)

In [26]:
t4 = Tensor([1, 2, 3, 4, 5])
t5 = (t4 + 1) * 2
t6 = (t5 * t4).relu().log_softmax()
print(t6.numpy())

[-56. -48. -36. -20.   0.]


In [29]:
class Linear:
    def __init__(self, in_features, out_features, bias=True, initialization: str='kaiming_uniform'):
        self.weight = getattr(Tensor, initialization)(out_features, in_features)
        self.bias = Tensor.zeros(out_features) if bias else None
    
    def __call__(self, x):
        return x.linear(self.weight.transpose(), self.bias)

In [30]:
class TinyNet:
    def __init__(self):
        self.l1 = Linear(784, 128, bias=False)
        self.l2 = Linear(128, 10, bias=False)
    def __call__(self, x):
        x = self.l1(x)
        x = x.leakyrelu()
        x = self.l2(x)
        return x

net = TinyNet()

In [35]:
def sparse_categorical_crossentropy(self, Y, ignore_index=-1) -> Tensor:
    loss_mask = Y != ignore_index
    y_counter = Tensor.arange(self.shape[-1], dtype=dtypes.int32, requires_grad=False, device=self.device).unsqueeze(0).expand(Y.numel(), self.shape[-1])
    y = ((y_counter == Y.flatten().reshape(-1, 1)).where(-1.0, 0) * loss_mask.reshape(-1, 1)).reshape(*Y.shape, self.shape[-1])
    return self.log_softmax().mul(y).sum() / loss_mask.sum()

In [32]:
from tinygrad.nn.optim import SGD

opt = SGD([net.l1.weight, net.l2.weight], lr=3e-4)

In [33]:
from extra.datasets import fetch_mnist

In [36]:
X_train, Y_train, X_test, Y_test = fetch_mnist()

with Tensor.train():
    for step in range(1000):
        samp = np.random.randint(0, X_train.shape[0], size=(64))
        batch = Tensor(X_train[samp], requires_grad=False)
        labels = Tensor(Y_train[samp])

        out = net(batch)

        loss = sparse_categorical_crossentropy(out, labels)

        opt.zero_grad()

        loss.backward()

        opt.step()

        pred = out.argmax(axis=-1)
        acc = (pred == labels).mean()

        if step % 100 == 0:
            print(f"Step {step+1} | Loss: {loss.numpy()} | Accuracy: {acc.numpy()}")

Step 1 | Loss: 133.93321228027344 | Accuracy: 0.0625
Step 101 | Loss: 12.861961364746094 | Accuracy: 0.671875
Step 201 | Loss: 3.878391981124878 | Accuracy: 0.84375
Step 301 | Loss: 6.062280654907227 | Accuracy: 0.859375
Step 401 | Loss: 2.115926504135132 | Accuracy: 0.890625
Step 501 | Loss: 1.229720115661621 | Accuracy: 0.953125
Step 601 | Loss: 2.6987829208374023 | Accuracy: 0.953125
Step 701 | Loss: 0.9411423802375793 | Accuracy: 0.921875
Step 801 | Loss: 2.1055243015289307 | Accuracy: 0.828125
Step 901 | Loss: 2.979787826538086 | Accuracy: 0.9375


In [38]:
with Timing("Time: "):
  avg_acc = 0
  for step in range(1000):
    # random sample a batch
    samp = np.random.randint(0, X_test.shape[0], size=(64))
    batch = Tensor(X_test[samp], requires_grad=False)
    # get the corresponding labels
    labels = Y_test[samp]

    # forward pass
    out = net(batch)

    # calculate accuracy
    pred = out.argmax(axis=-1).numpy()
    avg_acc += (pred == labels).mean()
  print(f"Test Accuracy: {avg_acc / 1000}")

Test Accuracy: 0.8896875
Time: 6915.36 ms
